# Table Extractor

This script takes pdfs of tables and turns them into usable tables.

In [ ]:
from collections import OrderedDict
from datetime import datetime
import csv, sys, os, io

import pyparsing as pp

## Functions & Parser Definitions

In [ ]:
def processXML(xml_name, out_name, row_description, header=""):
    table = pp.OneOrMore(pp.Group(row_description))
    with io.open(xml_name, 'r', encoding='utf8') as xml_file, io.open(out_name, 'w') as out_file:
        writer = csv.writer(out_file)
        print(writer)
        if header is not "":
            print(header)
            writer.writerow(header)
        data = xml_file.read()
        for result, start, end in table.scanString(data):
            for row in result:
                writer.writerow(row)

In [ ]:
decimals = pp.Combine(
    pp.Word(pp.nums, max=3)
    + pp.ZeroOrMore(pp.Suppress(",") + pp.Word(pp.nums, max=3))
    + pp.ZeroOrMore("." + pp.Word(pp.nums))
)
pct = pp.Combine(pp.Optional("-") + decimals + pp.Suppress("%"))
dollar = pp.Suppress("$") + decimals
date = pp.Combine(
    pp.Word(pp.nums, max=2)
    + "/"
    + pp.Word(pp.nums, max=2)
    + "/"
    + pp.Word(pp.nums, max=4)
)
cgr = pp.Word(pp.alphanums, exact=3)
territory = pp.Word(pp.nums, min=3, max=4)
zipcode = pp.Word(pp.nums, max=5)
gender = pp.Word("MF")
ypc = pp.Combine(pp.Word(pp.nums, exact=1) + pp.Suppress(pp.Optional("+")))

## Pre-processing
First, we'll use [tika](https://brewinstall.org/Install-tika-on-Mac-with-Brew/) to convert the pdf files into xml files so we can actually parse things.

In [ ]:
if not os.path.exists('../data/xml'):
    os.makedirs('../data/xml')

The below command works for a `brew`-installed version of `tika` on OS X. This command may vary depending on your version of `tika`'s installation and path.

In [ ]:
!tika -i ../data/pdf -o ../data/xml; cd ../data/xml; find . -type f -name '*.xml' -print0 | xargs -0 rename 's/\.pdf.xml/\.xml/'

## Tables

### CGR Definitions Table

In [ ]:
header = ['cgr', 'aa', 'bb' , 'cc', 'va', 'dd', 'hh', 'ss']
description = cgr + (decimals * 7)
fn = "cgr-definitions-table"
processXML("../data/xml/" + fn + ".xml", "../data/csv/" + fn + ".csv", description, header=header)

### CGR Premiums Table

In [ ]:
header = ['territory','gender','birthdate','ypc','current_premium','indicated_premium','selected_premium','underlying_premium','fixed_expenses','underlying_total_premium','cgr_factor','cgr']
premiums = pp.Combine(
    pp.Word(pp.nums)
    + pp.ZeroOrMore("." + pp.Word(pp.nums))
)
description = territory + gender + date + ypc + (premiums * 6) + decimals + cgr
fn = "cgr-premiums-table"
processXML("../data/xml/" + fn + ".xml", "../data/csv/" + fn + ".csv", description, header=header)

### Territory Definitions Table

In [ ]:
header = ['county','county_code','territory','zipcode','town', 'area']
numbers = pp.Word(pp.nums)
text = pp.Combine(
        pp.Word(pp.alphas.upper()) 
        + pp.ZeroOrMore(" " + pp.Word(pp.alphas.upper()))
    )
description = text + numbers + territory + zipcode + text + numbers
fn = "territory-definitions-table"
processXML("../data/xml/" + fn + ".xml", "../data/csv/" + fn + ".csv", description, header=header)